# MERGE COMPLET : TRAIN + STATIONS MÉTÉO + DONNÉES MÉTÉO

## DÉMARCHE:
1. Charger train.csv (données par région et semaine avec TauxGrippe)
2. Charger ListedesStationsMeteo.csv (liste des stations avec coordonnées)
3. Charger tous les fichiers synop (données météo par station et date)
4. Mapper chaque région à ses stations météo représentatives
5. Agréger les données météo par région et par semaine
6. Merger train.csv avec les données météo agrégées

## PROBLÈMES À RÉSOUDRE:
- train.csv : données par RÉGION et SEMAINE
- synop : données par STATION et DATE (horaire/journalier)
- Il faut :
  - a) Convertir les dates des fichiers synop en semaines
  - b) Mapper les stations aux régions
  - c) Agréger les données météo (moyenne par région/semaine)

In [ ]:
import pandas as pd
import numpy as np
import glob
from datetime import datetime
import warnings
warnings.filterwarnings('ignore')

print("="*80)
print("MERGE COMPLET : TRAIN + STATIONS MÉTÉO + DONNÉES MÉTÉO")
print("="*80)

## ÉTAPE 1 : Charger les données de base

In [ ]:
print("\n[1/6] Chargement des données de base...")

# Train.csv
df_train = pd.read_csv('data_origin/train.csv')
print(f"✓ train.csv : {df_train.shape}")
print(f"  Colonnes : {df_train.columns.tolist()}")
print(f"  Exemple :\n{df_train.head(3)}")

# Liste des stations météo
df_stations = pd.read_csv('data_origin/ListedesStationsMeteo.csv', sep=';')
print(f"\n✓ ListedesStationsMeteo.csv : {df_stations.shape}")
print(f"  Colonnes : {df_stations.columns.tolist()}")
print(f"  Exemple :\n{df_stations.head(3)}")

## ÉTAPE 2 : Créer le mapping Régions <-> Stations Météo

C'est la partie CRITIQUE du merge!

**Approche** : Chaque région française a des stations météo principales. On crée un mapping manuel basé sur la géographie.

**Note** : Les codes régions dans train.csv correspondent aux anciennes régions françaises (avant 2016)

In [ ]:
print("\n[2/6] Mapping Régions -> Stations Météo...")

# Mapping manuel : Région -> Liste de stations météo principales
REGION_STATION_MAPPING = {
    'ALSACE': ['07190', '07280'],  # Strasbourg
    'AQUITAINE': ['07510', '07630'],  # Bordeaux, Biarritz
    'AUVERGNE': ['07460', '07380'],  # Clermont-Ferrand, Vichy
    'BASSE-NORMANDIE': ['07027', '07139'],  # Caen
    'BOURGOGNE': ['07280', '07255'],  # Dijon
    'BRETAGNE': ['07110', '07117', '07130'],  # Brest, Rennes
    'CENTRE': ['07255', '07149'],  # Orléans, Tours
    'CHAMPAGNE-ARDENNE': ['07072', '07168'],  # Reims
    'CORSE': ['07761', '07790'],  # Ajaccio, Bastia
    'FRANCHE-COMTE': ['07299', '07280'],  # Besançon
    'HAUTE-NORMANDIE': ['07037', '07020'],  # Rouen, Le Havre
    'ILE-DE-FRANCE': ['07150', '07149'],  # Paris-Montsouris, Orly
    'LANGUEDOC-ROUSSILLON': ['07630', '07643'],  # Montpellier, Perpignan
    'LIMOUSIN': ['07434', '07335'],  # Limoges
    'LORRAINE': ['07090', '07180'],  # Nancy, Metz
    'MIDI-PYRENEES': ['07630', '07627'],  # Toulouse
    'NORD-PAS-DE-CALAIS': ['07005', '07015'],  # Lille
    'PAYS DE LA LOIRE': ['07222', '07130'],  # Nantes
    'PICARDIE': ['07005', '07015'],  # Amiens
    'POITOU-CHARENTES': ['07335', '07255'],  # Poitiers, La Rochelle
    "PROVENCE-ALPES-COTE D'AZUR": ['07650', '07690'],  # Marseille, Nice
    'RHONE-ALPES': ['07481', '07482'],  # Lyon
}

# Créer une table de mapping station -> région
station_region_map = []
for region, stations in REGION_STATION_MAPPING.items():
    for station in stations:
        station_region_map.append({
            'numer_sta': station,
            'region_name': region
        })

df_station_region = pd.DataFrame(station_region_map)
print(f"✓ Mapping créé : {len(df_station_region)} associations station-région")
print(f"  Régions couvertes : {df_station_region['region_name'].nunique()}")
print(f"  Exemple :\n{df_station_region.head(10)}")

## ÉTAPE 3 : Charger les données météo (SYNOP)

**Attention** : Les fichiers synop sont volumineux (~550MB au total)

**Optimisation** : On filtre immédiatement par les stations d'intérêt pour réduire la mémoire

In [ ]:
print("\n[3/6] Chargement des données météo (synop)...")

# Lister tous les fichiers synop
synop_files = sorted(glob.glob('DonneesMeteorologiques/DonneesMeteorologiques/synop.*.csv'))
print(f"✓ Fichiers synop trouvés : {len(synop_files)}")
if len(synop_files) > 0:
    print(f"  Premier : {synop_files[0].split('/')[-1]}")
    print(f"  Dernier : {synop_files[-1].split('/')[-1]}")

# Préparer les stations d'intérêt
stations_of_interest = df_station_region['numer_sta'].unique().tolist()
print(f"  Stations d'intérêt : {len(stations_of_interest)}")

# CORRECTION: Essayer les deux formats (string et int)
stations_int = [int(s) for s in stations_of_interest]
print(f"  Format string: {stations_of_interest[:3]}")
print(f"  Format int: {stations_int[:3]}")

synop_data_list = []
for i, file in enumerate(synop_files):
    if i % 12 == 0:  # Afficher progression tous les 12 mois
        print(f"  Chargement : {file.split('/')[-1]}...")
    
    try:
        df_synop = pd.read_csv(file, sep=';', low_memory=False)
        
        # CORRECTION: Essayer d'abord avec le format tel quel
        df_synop_filtered = df_synop[df_synop['numer_sta'].isin(stations_of_interest)]
        
        # Si rien trouvé, essayer avec int
        if len(df_synop_filtered) == 0:
            df_synop_filtered = df_synop[df_synop['numer_sta'].isin(stations_int)]
        
        if len(df_synop_filtered) > 0:
            synop_data_list.append(df_synop_filtered)
            
    except Exception as e:
        print(f"  ⚠ Erreur lecture {file} : {e}")

# Vérifier qu'on a des données
if len(synop_data_list) == 0:
    print("\n❌ ERREUR: Aucune donnée chargée!")
    print("   Causes possibles:")
    print("   1. Les IDs de station ne correspondent pas")
    print("   2. Les fichiers synop sont vides ou corrompus")
    print("   3. Les stations du mapping n'existent pas dans les fichiers synop")
    print("\n   → Vérifiez la cellule de débogage précédente!")
    raise ValueError("Aucune donnée synop chargée. Vérifiez le format des IDs de station.")

# Concaténer tous les fichiers
df_synop_all = pd.concat(synop_data_list, ignore_index=True)
print(f"\n✓ Données synop chargées : {df_synop_all.shape}")
print(f"  Colonnes : {len(df_synop_all.columns)}")
print(f"  Période : {df_synop_all['date'].min()} -> {df_synop_all['date'].max()}")
print(f"  Stations uniques : {df_synop_all['numer_sta'].nunique()}")

## ÉTAPE 2.5 : DÉBOGAGE - Vérifier le format des IDs de station

**IMPORTANT** : Les IDs peuvent être en format différent (string vs int, avec/sans padding)

In [ ]:
print("\n[3/6] Chargement des données météo (synop)...")

# Lister tous les fichiers synop
synop_files = sorted(glob.glob('DonneesMeteorologiques/DonneesMeteorologiques/synop.*.csv'))
print(f"✓ Fichiers synop trouvés : {len(synop_files)}")
print(f"  Premier : {synop_files[0].split('/')[-1]}")
print(f"  Dernier : {synop_files[-1].split('/')[-1]}")

# Filtrer par les stations d'intérêt
stations_of_interest = df_station_region['numer_sta'].unique().tolist()
print(f"  Stations d'intérêt : {len(stations_of_interest)}")

synop_data_list = []
for i, file in enumerate(synop_files):
    if i % 12 == 0:  # Afficher progression tous les 12 mois
        print(f"  Chargement : {file.split('/')[-1]}...")
    
    try:
        df_synop = pd.read_csv(file, sep=';', low_memory=False)
        # Filtrer uniquement les stations d'intérêt
        df_synop = df_synop[df_synop['numer_sta'].isin(stations_of_interest)]
        if len(df_synop) > 0:
            synop_data_list.append(df_synop)
    except Exception as e:
        print(f"  ⚠ Erreur lecture {file} : {e}")

# Concaténer tous les fichiers
df_synop_all = pd.concat(synop_data_list, ignore_index=True)
print(f"\n✓ Données synop chargées : {df_synop_all.shape}")
print(f"  Colonnes : {len(df_synop_all.columns)}")
print(f"  Période : {df_synop_all['date'].min()} -> {df_synop_all['date'].max()}")

## ÉTAPE 4 : Convertir les dates en semaines

**Problème** : synop a des dates au format AAAAMMJJHHMMSS, train.csv a des semaines au format AAAASS

**Solution** : Utiliser ISO calendar pour extraire l'année et le numéro de semaine

In [ ]:
print("\n[5/6] Agrégation des données météo par région et semaine...")

# CORRECTION: S'assurer que les IDs sont au même format pour le merge
# Convertir numer_sta en string dans les deux DataFrames
df_synop_all['numer_sta'] = df_synop_all['numer_sta'].astype(str).str.zfill(5)
df_station_region['numer_sta'] = df_station_region['numer_sta'].astype(str).str.zfill(5)

# Merger avec le mapping station->région
df_synop_all = df_synop_all.merge(df_station_region, on='numer_sta', how='inner')
print(f"✓ Merge avec mapping : {df_synop_all.shape}")

if len(df_synop_all) == 0:
    print("\n❌ ERREUR: Le merge a échoué! Aucune correspondance trouvée.")
    print("   → Vérifiez le format des IDs dans la cellule de débogage")
    raise ValueError("Merge échoué entre synop et mapping stations")

# Sélectionner les variables météo importantes
meteo_vars = [
    'tend', 'dd', 'ff', 't', 'td', 'u', 'vv', 'n', 'nbas', 'hbas',
    'pres', 'niv_bar', 'geop', 'tend24', 'tn12', 'tn24', 'tx12', 'tx24',
    'tminsol', 'tw', 'raf10', 'rafper', 'per', 'ht_neige', 'ssfrai',
    'perssfrai', 'rr1', 'rr3', 'rr6', 'rr12', 'rr24'
]

# Vérifier quelles variables existent
meteo_vars_available = [v for v in meteo_vars if v in df_synop_all.columns]
print(f"  Variables météo disponibles : {len(meteo_vars_available)}/{len(meteo_vars)}")

# Convertir les variables météo en numérique (remplacer 'mq' par NaN)
for var in meteo_vars_available:
    df_synop_all[var] = pd.to_numeric(df_synop_all[var], errors='coerce')

# Agréger par région et semaine (moyenne)
agg_dict = {var: 'mean' for var in meteo_vars_available}
df_meteo_agg = df_synop_all.groupby(['region_name', 'week_year'], as_index=False).agg(agg_dict)

print(f"✓ Agrégation effectuée : {df_meteo_agg.shape}")
print(f"  Régions uniques : {df_meteo_agg['region_name'].nunique()}")
print(f"  Semaines uniques : {df_meteo_agg['week_year'].nunique()}")
print(f"\n  Exemple :\n{df_meteo_agg.head(3)}")

## ÉTAPE 5 : Agréger les données météo par région et semaine

**Approche** :
1. Merger synop avec le mapping station->région
2. Grouper par (région, semaine)
3. Calculer la moyenne de chaque variable météo

In [ ]:
print("\n[5/6] Agrégation des données météo par région et semaine...")

# Merger avec le mapping station->région
df_synop_all = df_synop_all.merge(df_station_region, on='numer_sta', how='inner')
print(f"✓ Merge avec mapping : {df_synop_all.shape}")

# Sélectionner les variables météo importantes
meteo_vars = [
    'tend', 'dd', 'ff', 't', 'td', 'u', 'vv', 'n', 'nbas', 'hbas',
    'pres', 'niv_bar', 'geop', 'tend24', 'tn12', 'tn24', 'tx12', 'tx24',
    'tminsol', 'tw', 'raf10', 'rafper', 'per', 'ht_neige', 'ssfrai',
    'perssfrai', 'rr1', 'rr3', 'rr6', 'rr12', 'rr24'
]

# Vérifier quelles variables existent
meteo_vars_available = [v for v in meteo_vars if v in df_synop_all.columns]
print(f"  Variables météo disponibles : {len(meteo_vars_available)}/{len(meteo_vars)}")

# Convertir les variables météo en numérique (remplacer 'mq' par NaN)
for var in meteo_vars_available:
    df_synop_all[var] = pd.to_numeric(df_synop_all[var], errors='coerce')

# Agréger par région et semaine (moyenne)
agg_dict = {var: 'mean' for var in meteo_vars_available}
df_meteo_agg = df_synop_all.groupby(['region_name', 'week_year'], as_index=False).agg(agg_dict)

print(f"✓ Agrégation effectuée : {df_meteo_agg.shape}")
print(f"  Exemple :\n{df_meteo_agg.head(3)}")

## ÉTAPE 6 : Merge final avec train.csv

**Clés de jointure** :
- region_name (normalisé en majuscules)
- week (de train.csv) = week_year (de synop agrégé)

In [ ]:
print("\n[6/6] Merge final avec train.csv...")

# Normaliser les noms de régions pour le merge
df_train['region_name_clean'] = df_train['region_name'].str.upper().str.strip()
df_meteo_agg['region_name_clean'] = df_meteo_agg['region_name'].str.upper().str.strip()

# Merger
df_final = df_train.merge(
    df_meteo_agg,
    left_on=['region_name_clean', 'week'],
    right_on=['region_name_clean', 'week_year'],
    how='inner'
)

print(f"✓ Merge effectué : {df_final.shape}")
print(f"  Colonnes : {len(df_final.columns)}")

# Nettoyer les colonnes dupliquées
cols_to_drop = ['region_name_clean', 'week_year', 'region_name_y']
df_final = df_final.drop(columns=[c for c in cols_to_drop if c in df_final.columns])

# Renommer region_name_x en region_name
if 'region_name_x' in df_final.columns:
    df_final = df_final.rename(columns={'region_name_x': 'region_name'})

## Sauvegarde du fichier final

In [ ]:
output_file = 'data_plus/train_weather_merged_complete.csv'
df_final.to_csv(output_file, index=False)

print(f"\n{'='*80}")
print("MERGE TERMINÉ AVEC SUCCÈS!")
print("="*80)
print(f"\nFichier généré : {output_file}")
print(f"  Dimensions : {df_final.shape}")
print(f"  Observations : {len(df_final)}")
print(f"  Variables météo ajoutées : {len(meteo_vars_available)}")
print(f"\nAperçu final :")
print(df_final.head(3))

print(f"\nColonnes finales ({len(df_final.columns)}) :")
print(df_final.columns.tolist())

print(f"\n✓ Le fichier est prêt pour l'analyse et la modélisation!")

## Analyse du résultat

Vérifions la couverture et la qualité du merge

In [ ]:
print("\nAnalyse du merge :")
print(f"  train.csv original : {len(df_train)} lignes")
print(f"  Après merge : {len(df_final)} lignes")
print(f"  Taux de couverture : {len(df_final)/len(df_train)*100:.2f}%")

print(f"\nRégions couvertes :")
print(df_final['region_name'].value_counts())

print(f"\nPériode couverte :")
print(f"  Semaines : {df_final['week'].min()} -> {df_final['week'].max()}")

print(f"\nValeurs manquantes :")
missing = df_final.isnull().sum()
missing = missing[missing > 0].sort_values(ascending=False)
print(missing.head(10))